In [49]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

In [50]:
data = pd.read_csv('data_final.csv')
# Keeping only the neccessary columns
data = data[['Headline','Price movement']]

In [51]:
data = data[data['Price movement'] != "#N/A"]
data['Headline'] = data['Headline'].apply(lambda x: x.lower())
data['Headline'] = data['Headline'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

print(data[data['Price movement'] == 'Up'].size)
print(data[data['Price movement'] == 'Down'].size)

for idx,row in data.iterrows():
    row[0] = row[0].replace('rt',' ')

max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['Headline'].values)
X = tokenizer.texts_to_sequences(data['Headline'].values)
X = pad_sequences(X)

6040
4916


In [52]:
from keras.layers import Conv1D, MaxPooling1D, Flatten

embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(Conv1D(filters=128, kernel_size=4, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=64, kernel_size=4, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=32, kernel_size=4, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(2, activation='sigmoid'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_7 (Embedding)     (None, 35, 128)           256000    
                                                                 
 conv1d_12 (Conv1D)          (None, 35, 128)           65664     
                                                                 
 max_pooling1d_9 (MaxPooling  (None, 17, 128)          0         
 1D)                                                             
                                                                 
 conv1d_13 (Conv1D)          (None, 17, 64)            32832     
                                                                 
 max_pooling1d_10 (MaxPoolin  (None, 8, 64)            0         
 g1D)                                                            
                                                                 
 conv1d_14 (Conv1D)          (None, 8, 32)            

In [53]:
Y = pd.get_dummies(data['Price movement']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(3778, 35) (3778, 2)
(1862, 35) (1862, 2)


In [54]:
batch_size = 128
model.fit(X_train, Y_train, epochs=7, batch_size=batch_size, verbose = 2)

Epoch 1/7
30/30 - 2s - loss: 0.6695 - accuracy: 0.5429 - 2s/epoch - 58ms/step
Epoch 2/7
30/30 - 1s - loss: 1.2004 - accuracy: 0.5061 - 661ms/epoch - 22ms/step
Epoch 3/7
30/30 - 1s - loss: 17.4116 - accuracy: 0.5011 - 602ms/epoch - 20ms/step
Epoch 4/7
30/30 - 1s - loss: 277.3576 - accuracy: 0.4995 - 674ms/epoch - 22ms/step
Epoch 5/7
30/30 - 1s - loss: 2675.6853 - accuracy: 0.5003 - 667ms/epoch - 22ms/step
Epoch 6/7
30/30 - 1s - loss: 9969.4814 - accuracy: 0.5061 - 684ms/epoch - 23ms/step
Epoch 7/7
30/30 - 1s - loss: 25386.3203 - accuracy: 0.5159 - 684ms/epoch - 23ms/step


In [55]:
# Final evaluation of the model
scores = model.evaluate(X_test, Y_test, verbose=2)
print("Accuracy: %.2f%%" % (scores[1]*100))

59/59 - 0s - loss: 1076.1598 - accuracy: 0.5166 - 318ms/epoch - 5ms/step
Accuracy: 51.66%
